<a href="https://colab.research.google.com/github/Nancy-Shi/Complex_Networks/blob/main/070323_Rumor_Basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import random
import math as math
from math import log

In [9]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [10]:
def generate_degree_sequence(n, gamma, kmin):
    # Generate a random set from the power law distribution
    u = np.random.uniform(size=n)
    degrees = np.ceil((1.0 - u) ** (-1.0 / (gamma - 1.0)))

    # Adjust degrees based on the minimum and maximum degree values
    kmax = int(1.5*n**(1/4)) # max degree allowed is 1.5*n^(1/4)
    degrees = degrees[(degrees >= kmin) & (degrees <= kmax)].astype(int)

    # Truncate or pad the sequence to match the length specified
    if len(degrees) >= n:
        degrees = degrees[:n]
    else:
        degrees = np.concatenate((degrees, np.full(n - len(degrees), kmin)))

    return degrees.tolist()

In [11]:
def generate_configuration_model(degree_sequence):
    # Create an empty graph and sum up the degrees
    G = nx.Graph()
    degree_sum = sum(degree_sequence)

    # Print error message if the sum is odd
    if degree_sum % 2 != 0:
        raise ValueError("Degree sum is odd")

    # Create a list of nodes based on the degree sequence
    nodes = []
    for node, degree in enumerate(degree_sequence):
        nodes.extend([node] * degree)

    # Shuffle the nodes list
    random.shuffle(nodes)

    # Track visited edges
    existing_edges = set()

    # Pair the nodes and add edges to the graph
    for i in range(0, len(nodes), 2):
        node1 = nodes[i]
        node2 = nodes[i + 1]

        # Check for self-edges and duplicate edges
        if node1 != node2 and (node1, node2) not in existing_edges:
            G.add_edge(node1, node2)
            existing_edges.add((node1, node2))
            existing_edges.add((node2, node1))

    return G

In [24]:
## The following is the baseline rumor model
## There are 3 states: S(ignorant), I(spreader), R(stifler)

def Rumor_Basic(nw, lam, alp, n_samp):
   # initialize variables
    t_max = 10000
    n_times = 100000  # number of time points at which rho_R is recorded and averaged
    rho_R_av = np.zeros(n_times+1)  # Initialize rho_R_av as a numpy array of zeros
    t_sav = np.zeros(n_times+1)  # Initialize t_sav as a numpy array of zeros
    t_sav0 = 0
    N = len(nw.nodes)
    degrees = dict(nw.degree())
    kmax = max(degrees.values())

    t_end = np.zeros(n_samp+1)

    # Initialize states with ignorant individuals
    states = {j: 0 for j in nw.nodes()}

    for i_samp in range(1, n_samp+1):
        t = 0
        n_sav = 0 # an index variable that keeps track of the current position in the rho_R_av array.
        t_sav[n_sav] = t_sav0
        rho_R = 0

        # Randomly select an initial spreader node
        inf = []
        #rec = []
        initial_node = np.random.choice(nw.nodes())
        states[initial_node] = 1
        inf.append(initial_node)
        N_inf = 1
        N_rec = 0

        while t < t_max and N_inf > 0:
            total_rate = alp * N_inf + 2 * lam * N_rec
            tau = -np.log(np.random.uniform(1e-6, 1)) / total_rate
            t += tau

            # Update average rho_R at specific time points
            while n_sav < n_times and t > t_sav[n_sav]:
                rho_R_av[n_sav] += rho_R
                n_sav += 1
                t_sav[n_sav] += t_max/n_times

            # Determine which event occurs
            event = np.random.uniform()
            p1 = (alp * N_inf) / total_rate
            p2 = (alp * N_inf + lam * N_rec)/total_rate

            # Determine if accept selected spreader based on degree distribution
            q_deg = np.random.uniform()

            if event > p1: # Recovery
                # Select a spreader individual to recover
                rec_node = np.random.choice(inf)
                if q_deg > degrees[rec_node]/kmax:
                    # Select a random neighbor to contact
                    neighbors = list(nw.neighbors(rec_node))
                    if len(neighbors) > 0:
                      # Recover event 1:  I + R --> 2R
                      if event > p2:
                        neighbor = np.random.choice(neighbors)
                        if states[neighbor == 2]:
                            # Update spreader to stifler if the selected neighbor is a stifler
                            states[rec_node] = 2
                            N_rec += 1
                            N_inf -= 1
                            #rec.append(rec_node)
                            inf.remove(rec_node)
                            rho_R += (1 / N)

                      else: # Recovery event 2: I + I --> R + I
                        neighbor = np.random.choice(neighbors)
                        if states[neighbor == 1]:
                            # Update spreader to stifler if the selected neighbor is a spreader
                            states[rec_node] = 2
                            N_rec += 1
                            N_inf -= 1
                            #rec.append(rec_node)
                            inf.remove(rec_node)
                            rho_R += (1 / N)


            else: # Infection event: I + S --> 2I
                # Select a spreader individual to spread the rumor
                inf_node = np.random.choice(inf)
                if q_deg > degrees[inf_node]/kmax:
                  # Select a random neighbor to contact
                  neighbors = list(nw.neighbors(inf_node))
                  if len(neighbors) > 0:
                    neighbor = np.random.choice(neighbors)
                    # Check if the selected neighbor is ignorant
                    if states[neighbor] == 0:
                        states[neighbor] = 1  # ignorant neighbor becomes spreader
                        inf.append(neighbor)
                        N_inf += 1

        t_end[i_samp] = t

        # Reset spreader and stiflers for the next sample
        for node, state in states.items():
            if state == 1 or state == 2:
                states[node] = 0

    # Normalize the average rho_R values by dividing by the number of samples
    rho_R_av /= n_samp
    t_end_av = sum(t_end)/n_samp

    return t_sav, rho_R_av, t_end_av


In [ ]:
n= 2000
gamma = 4.0
kmin = 3
degrees = generate_degree_sequence(n, gamma, kmin)
print(degrees)
nw = generate_configuration_model(degrees)

alp = 1

n_samp = 10

lambda_values = np.linspace(0, 1, num=20)  # Generate lambda values from 0 to 1 with 0.05 steps
rho_R_av_values = []
t_end_av_values = []

for lam in lambda_values:
    time, rho_R_av, t_end_av = Rumor_Basic(nw, lam, alp, n_samp)
    rho_R_av_values.append(rho_R_av[-2])  # Append the last value of rho_R_av to the list
    t_end_av_values.append(t_end_av)

# Convert the lists to numpy arrays
avg_rho_R_values = np.array(rho_R_av_values)
avg_t_end_values = np.array(t_end_av_values)

# Plot the two-panel graph
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(8, 10))

# Top panel: average density of stifler rho_R against spreading rate lambda
ax1.plot(lambda_values, avg_rho_R_values, 'o-')
ax1.set_xlabel('Lambda')
ax1.set_ylabel('Average Stifler Density')
ax1.set_title('Average rho_R vs. Lambda')
ax1.legend()

# Bottom panel: average rumor die-out time t_max against spreading rate lambda
ax2.plot(lambda_values, avg_t_end_values, 'o-')
ax2.set_xlabel('Lambda')
ax2.set_ylabel('Average Rumor Die-Out Time')
ax2.set_title('Average t_max vs. Lambda')
ax2.legend()

# Adjust spacing between subplots
plt.subplots_adjust(hspace=0.4)

# Display the graph
plt.show()

[3, 3, 4, 3, 3, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 6, 6, 3, 3, 3, 3, 3, 5, 6, 3, 3, 3, 5, 3, 3, 4, 3, 3, 4, 5, 5, 3, 3, 3, 3, 3, 3, 3, 7, 3, 3, 3, 3, 4, 4, 4, 4, 3, 3, 3, 7, 5, 4, 3, 3, 3, 3, 3, 3, 3, 4, 3, 3, 6, 3, 3, 4, 3, 3, 3, 3, 3, 3, 4, 3, 3, 3, 5, 5, 4, 6, 3, 6, 3, 4, 3, 5, 4, 3, 3, 4, 3, 3, 3, 3, 3, 5, 4, 3, 3, 4, 3, 3, 3, 3, 5, 4, 3, 6, 3, 4, 3, 4, 3, 3, 3, 4, 4, 7, 3, 3, 4, 3, 3, 3, 3, 3, 3, 3, 4, 5, 3, 6, 3, 3, 3, 3, 6, 3, 3, 3, 3, 8, 3, 3, 3, 3, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 3, 3, 3, 3, 3, 4, 3, 5, 6, 3, 4, 4, 5, 3, 3, 10, 5, 3, 3, 4, 4, 5, 3, 4, 3, 3, 5, 3, 3, 3, 3, 3, 3, 5, 8, 6, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,